In [76]:
from transformers import DistilBertModel, DistilBertTokenizer
from torch.utils.data.dataset import Dataset
import torch
from modules.tweet_data import read_raw_data
import re

In [9]:
MODEL_NAME = 'distilbert-base-cased'
# Load pre-trained model
model = DistilBertModel.from_pretrained(MODEL_NAME)
# Load pre-trained model tokenizer (vocabulary)
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

In [66]:
#tweet_df = read_raw_data()
pearkes = tweet_df[tweet_df.handle=='pearkes']

In [67]:
REGEX_DICT = {'link': "http\S+",
              'piclink': "pic.twitter.com\S+",
              'hashtag': "\B#\w*[a-zA-Z]+\w*|#\w*[a-zA-Z]+\w*",
              'email': "[a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+",
              'add': "\s([@][\w_-]+)|[@][\w_-]+"}

for key, value in REGEX_DICT.items():
    print('Filtering', key)
    pearkes.tweet = pearkes.tweet.apply(lambda tweet: re.sub(value, '', tweet))

Filtering link
Filtering piclink
Filtering hashtag
Filtering email
Filtering add


In [70]:
tokenizer.tokenize(pearkes.iloc[101001].tweet)

['yes', ',', 'exactly', '.', '.', '.', 'that', 'or']

In [71]:
data = tokenizer.batch_encode_plus(pearkes.tweet, 
                            return_tensors="pt",
                            padding=True,
                            truncation=True)

In [77]:
class MarketSentimentDataset(Dataset):
    def __init__(self, tweets, targets, tokenizer):
        self.tweets = reviews
        self.targets = targets
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.tweets)
    def __getitem__(self, item):
        tweet = str(self.tweets[item])
        target = self.targets[item]
        tokenized = self.tokenizer.encode_plus(tweet, 
                                             return_tensors="pt",
                                             padding=True,
                                             truncation=True)
        return {
          'review_text': tweet,
          'input_ids': tokenized['input_ids'].flatten(),
          'attention_mask': tokenized['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }